In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import zipfile
import os
from PIL import Image

class ECGGenerator3D(nn.Module):
    def __init__(self, input_dim, output_channels, base_dim):
        super(ECGGenerator3D, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, base_dim * 4 * 4 * 4),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (base_dim, 4, 4, 4)),
            nn.ConvTranspose3d(base_dim, base_dim // 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(base_dim // 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(base_dim // 2, base_dim // 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(base_dim // 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(base_dim // 4, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

def extract_ecg_data(zip_path, extract_to="ecg_data"):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    ecg_images = []
    for root, _, files in os.walk(extract_to):
        for file_name in files:
            if file_name.endswith('.png'):
                file_path = os.path.join(root, file_name)
                # Open the image and convert to grayscale
                image = Image.open(file_path).convert('L')

                # Resize to reduce memory usage
                image = image.resize((128, 128))

                # Convert to NumPy array with reduced precision
                image_array = np.array(image, dtype=np.float32) / 255.0
                ecg_images.append(image_array)

    if len(ecg_images) == 0:
        raise FileNotFoundError("No valid ECG image file found in the zip.")

    # Stack the images after processing all
    return np.stack(ecg_images)

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    input_dim = 100
    output_channels = 1
    base_dim = 64
    zip_path = "C:\\Users\\KIIT\\OneDrive\\Desktop\\archive (1).zip"
    try:
        raw_ecg_data = extract_ecg_data(zip_path)
    except FileNotFoundError as e:
        print(e)
        return
    print("Raw ECG Data Shape:", raw_ecg_data.shape)
    generator = ECGGenerator3D(input_dim, output_channels, base_dim).to(device)
    z = torch.randn((1, input_dim)).to(device)
    generated_data = generator(z)
    print("Generated 3D Data Shape:", generated_data.shape)
    generated_data = generated_data.cpu().detach().numpy()
    print("Generated Data (First slice, first 10 values):", generated_data[0, 0, 0, :10])

if __name__ == "__main__":
    main()
#code needs higher cpu and gpu, implemented in kiit lab